In [58]:
from IPython.display import display, HTML
from functools import reduce 
import pandas as pd 
import datetime, calendar
from pyspark.sql.types import ArrayType, MapType, DataType, StringType, StructType
from pyspark.sql.functions import explode, split, json_tuple, udf, desc, asc, lower, lit, collect_set, count, regexp_extract
import pandas as pd
import time
from datetime import datetime

import sys
### This is no longer required in order to load the btn library
### It is now installed as a first class python package citizen
# sys.path.append('/home/d2dcrc/btn_library/')

### A normal import is all that is required
from btndal.btn_datascience import *

pd.set_option("display.max_colwidth", 1000)

%load_ext autoreload
%autoreload 2

### Use the library as normal
### See https://d2dcrc.atlassian.net/wiki/display/BTN/Data+Access+Documention for more usage information
btn_connect(sc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using Python 3.5.3 | packaged by conda-forge | (default, Jan 23 2017, 19:01:48) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
Using Spark 1.6.3
Application ID is application_1501728610222_6409

Monitoring/Debugging URLs. These require a VPN to access
YARN Tracking URL: http://yarn-resourcemanager.d2dcrc.net:8088/cluster/app/application_1501728610222_6409/
Spark UI http://yarn-resourcemanager.d2dcrc.net:8089/proxy/application_1501728610222_6409/
History http://yarn-resourcemanager.d2dcrc.net:18080/history/application_1501728610222_6409



In [59]:
#year = 2017
#month = 6
#day = 14
#tweetsDF = btn_twitter_by_date(sqlContext, publicationYear=year, publicationMonth=month, publicationDay=day)
#tweetsDF.count()

In [60]:
publicationYear = 2017
publicationMonth = 5
publicationDay = 1
 
datapath = 'hdfs:///training/signals-mvpoc/freeport/documents/gnip_v0.1_by_local_date.json/publicationYear='+ str(publicationYear) #+'/publicationMonth='+ str(publicationMonth) +'/publicationDay=' + str(publicationDay)  
tweetsDF = sqlContext.read.json(datapath)



In [61]:
tweetsDF.printSchema()

root
 |-- actorEntities: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- end: long (nullable = true)
 |    |    |-- start: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- authorGeo: struct (nullable = true)
 |    |-- candidateMetadataJson: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- countryCode: string (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- lon: double (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- postcode: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- authorInfo: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- displayName: string (nullable = true)
 |    |    |-- favouritedPostCount: long (nullable = true)
 |    |    |-- followerCount: long (nullable = true)
 |    |    |-- followingCount

In [71]:
pdTweets = tweetsDF.select("publicationTimestamp","authorInfo.id","text", "authorGeo.country").orderBy("publicationTimestamp")
pd.set_option('display.max_colwidth', 500)
#display.display(pdTweets.toPandas())

In [72]:
newpd = pdTweets.toPandas()

In [73]:
newpd

,publicationTimestamp,id,text,country
0,1483230142,[id:twitter.com:730978561394380800],Wings topped Synnara year end charts followed by HYYH YF in 5th place and HYYH p1 and 2 are still charting after + year of their release https://t.co/cRq93ptFii,Australia
1,1483230637,[id:twitter.com:2465291],2016 will be hard to beat in terms of time spent on bikes. My standing goal for the last 3 years… https://t.co/NJLPtFI5My,Australia
2,1483234909,[id:twitter.com:790168492808953856],Telkomsel: Jihanluvazzira Daerah DKI Jakarta sudah ter-cover jaringan 4G LTE Telkomsel. Info area https://t.co/VHptfIGmnU :) -Kiran,None
3,1483243379,[id:twitter.com:1960006242],"Check out ""All Flaws "" by John Revitte - https://t.co/Rf256ptfI4 @BearsLegacy",Australia
4,1483260935,[id:twitter.com:703414324363030532],iqbalwidyawan Waru - Juanda &amp; Surabaya - Gresik Bpk Iqbal.06 Tks :) ^Olid,Australia
5,1483263701,[id:twitter.com:79004617],@korantempo @Carl_C_Icahn owns almost 9 % of freeport..he threw jim bob under the bus..maybe we can reason &amp; strike a better deal with him..,Indonesia
6,1483264402,[id:twitter.com:369398828],Cari angin sama anak kesayangan! 😆 (at Jl. Jakarta (I Love GKB) Gresik) [pic] — https://t.co/8zM09wbqaA,Indonesia
7,1483266655,[id:twitter.com:2903459562],I liked a @YouTube video https://t.co/adsjPTFI2v Mama Mia part 6,Australia
8,1483274603,[id:twitter.com:1290140503],Visit Live cam girls 24/7\n\nTotaly free to signup\n\nhttps://t.co/Vm3zjYObi7 https://t.co/pr1PtfIHrI,Australia
9,1483303252,[id:twitter.com:3278207173],https://t.co/1WptfIiieZ,الإمارات العربيّة المتّحدة


In [74]:
keywords = ["protest", "muslim", "islam", "outcry", "asylum", "organisation", "threat", "union", "centrelink",
            "opposition", "parade", "council",
            "federal", "strike", "harass", "refugee", "riot", "community", "reclaim", "poster", "demonstration",
            "petition", "funding", "barrier", "march",
            "crowd", "celebration", "action", "barricade", "placard", "gather", "resident", "patriot", "bigot",
            "racism", "national", "decision", "movement",
            "mentality", "racist", "agency", "mosque", "highlight", "halaal", "turmoil", "activist", "disturbance",
            "victory", "equality", "blockade", "anger",
"ideal", "unite", "extremist", "anzac", "rally", "culture", "unrest", "terror", "terrorist"
, "prevent", "cpsu", "anger", "concern","stoppage", "park","value", "claim","immigrant","opponent",
                 "standoff", "spray","sign","urban","pressure","destroy","side", "unrest", "stoking" ,
            "perusuhan",   "tunjuk perasaan",    "unjuk perasaan",    "unjuk rasa buruh",    "demo",    "berdemo",    "demonstrasi",
"kegaduhan",    "kegemparan",    "keributan",    "kericuhan",    "kerisauan",    "kerusuhan",    "meledakkan",
"memberlakukan jam malam",    "mengganggu", "menggerebakkan",  "merayakan",  "penampilan", 
            "mentalitas", "rasis", "agensi", "masjid", "sorotan", "halal", "kekacauan", "aktivis", "gangguan"
            "kemenangan", "persamaan", "blokade", "kemarahan",
"ideal", "bersatu", "ekstremis", "anzac", "rally", "culture", "kerusuhan", "teror", "teroris"
"mencegah", "cpsu", "kemarahan", "kekhawatiran", "penghentian", "taman", "nilai", "klaim", "imigran", "lawan"
                 "kebuntuan", "semprotan", "tanda", "perkotaan", "tekanan", "hancurkan", "sisi", "kerusuhan", "menyalakan"
           ]


#keywords_indoenesian= ["protes", "muslim", "islam", "protes", "suaka", "organisasi", "ancaman", "persatuan", "sentrelink"
#            "oposisi", "parade", "dewan",
#            "federal", "pemogokan", "pelecehan", "pengungsi", "kerusuhan", "komunitas", "reklamasi", "poster", "demonstrasi"
#            "petisi", "pendanaan", "penghalang", "maret",
#            "kerumunan", "perayaan", "tindakan", "barikade", "plakat", "kumpulkan", "penduduk", "patriot", "fanatik"
#            "rasisme", "nasional", "keputusan", "gerakan", 
#"perusuhan",   "tunjuk perasaan",    "unjuk perasaan",    "unjuk rasa buruh",    "demo",    "berdemo",    "demonstrasi",
#"kegaduhan",    "kegemparan",    "keributan",    "kericuhan",    "kerisauan",    "kerusuhan",    "meledakkan",
#"memberlakukan jam malam",    "mengganggu", "menggerebakkan",  "merayakan",  "penampilan"  ]




In [75]:
import pandas as pd
import datetime as dt
import multiprocessing as mp
import time 
startTime = time.time()


def matches_util(args):
    df_in, word_list, min_keywords = args

    # Check for every keyword in ever row
    values = pd.DataFrame()
    for kw in word_list:
        values[kw] = df_in.text.str.contains(kw, case=False)

    # Add keyword counts
    values["wordCnt"] = [v.sum() for k, v in values.iterrows()]

    # If keyword count is greater than min_keywords then include in result
    return df_in.loc[values.wordCnt >= min_keywords]


def get_matches(df_in, word_list, min_keywords=2, output_filepath="", multi_threads=False, thread_cnt=2):
    print("Looking for matches.\nmin_keywords={}\noutput_filepath={}\nmulti_threads={}\nword_list={}".format(
        min_keywords,
        output_filepath,
        multi_threads,
        word_list))

    # Make sure bodyText field exists
    assert "text" in df_in.columns, "Field 'text' was not found in input DataFrame!"
    ts1 = dt.datetime.now()
    result = None
    resList = None

    # Use multi-threading if enabled
    if multi_threads:
        with mp.Pool(thread_cnt) as p:
            threadCnt = thread_cnt
            bins = pd.cut(df_in.index, threadCnt, labels=range(threadCnt))
            resList = p.map(matches_util, ((df_in.loc[bins == i], keywords, min_keywords) for i in range(thread_cnt)))

    # Else process without multi-threading
    else:
        result = matches_util((df_in, keywords, min_keywords))

    # Get results from queue
    if result is None:
        result = pd.concat(resList)

    ft = dt.datetime.now() - ts1
    print("Finished looking for matches.  Time Elapsed={:.1f}s".format(ft.total_seconds()))

    # Save resulting matches to file if a filepath was passed
    if output_filepath != "":
        result.to_csv(output_filepath)
        print("Saved output to {}".format(output_filepath))

    return result

In [76]:
get_matches(newpd, word_list=keywords, min_keywords=2, output_filepath="oneday.csv", multi_threads=True, thread_cnt=8)

Looking for matches.
min_keywords=2
output_filepath=oneday.csv
multi_threads=True
word_list=['protest', 'muslim', 'islam', 'outcry', 'asylum', 'organisation', 'threat', 'union', 'centrelink', 'opposition', 'parade', 'council', 'federal', 'strike', 'harass', 'refugee', 'riot', 'community', 'reclaim', 'poster', 'demonstration', 'petition', 'funding', 'barrier', 'march', 'crowd', 'celebration', 'action', 'barricade', 'placard', 'gather', 'resident', 'patriot', 'bigot', 'racism', 'national', 'decision', 'movement', 'mentality', 'racist', 'agency', 'mosque', 'highlight', 'halaal', 'turmoil', 'activist', 'disturbance', 'victory', 'equality', 'blockade', 'anger', 'ideal', 'unite', 'extremist', 'anzac', 'rally', 'culture', 'unrest', 'terror', 'terrorist', 'prevent', 'cpsu', 'anger', 'concern', 'stoppage', 'park', 'value', 'claim', 'immigrant', 'opponent', 'standoff', 'spray', 'sign', 'urban', 'pressure', 'destroy', 'side', 'unrest', 'stoking', 'perusuhan', 'tunjuk perasaan', 'unjuk perasaan', 

,publicationTimestamp,id,text,country
15,1483349234,[id:twitter.com:807229020664262656],Dishub DKI Nilai Pelabuhan di Jakarta Sudah Penuhi Standar https://t.co/IaV9ptFIrK #RepzoneKalTim #Repzone,Indonesia
46,1483400067,[id:twitter.com:733844080950542336],"BOTTOM LINE: If you hate others bc of the color of thier skin, religion, origin, gender, sexual orientation, etc, YOU ARE NOT A PATRIOT! https://t.co/Lm1uOptFi0",Australia
61,1483438934,[id:twitter.com:806578411674234880],"TYSONS, Va., Jan. 3, 2017 /PRNewswire-USNewswire/ -- Pentagon Federal Credit Union will change the lives of more... https://t.co/ptfinyYSQD",None
77,1483574246,[id:twitter.com:35606154],Inside Donald Trump's Extremist Education Agenda @alternet https://t.co/ptFiwIlRKr,United States of America
242,1483755911,[id:twitter.com:552821881],Paul Ryan Busts Out Laughing As Democrats Protest Trump’s Election Certification [VIDEO] https://t.co/jmXfrPtFIA via @dailycaller,United States of America
301,1483964803,[id:twitter.com:702538765982670848],@dptfinance can I ask why the Union Flag has been removed late this morning from the Flagpole outside Clare House?,UK
304,1483974057,[id:twitter.com:835234932],"This Tube Strike is just not right, unions knew that LU would close ticket offices eventually with smart ticketing. #TubeStrike https://t.co/TpwNNpTFIw",UK
396,1484243605,[id:twitter.com:1954280094],"In the news: Snowshoe Demonstration at Hedgehog Mountain Trails, Freeport https://t.co/oohWkNa162",United States of America
397,1484244084,[id:twitter.com:4848867712],"Snowshoe Demonstration at Hedgehog Mountain Trails, Freeport https://t.co/wcB0RblPEB",None
399,1484245351,[id:twitter.com:172379107],"Snowshoe Demonstration at Hedgehog Mountain Trails, Freeport https://t.co/FdZytx9GjN",United States of America


In [78]:
import numpy as np
import pandas as pd
import csv

ds = pd.read_csv("oneday.csv") #loads the csv correctly

In [79]:
adl2= ds.where(ds['country'].str.contains('Indonesia')).dropna()
adl2.to_csv('indo_sample.csv')

In [80]:
adl2

,Unnamed: 0,publicationTimestamp,id,text,country
0,15.0,1.483349e+09,['id:twitter.com:807229020664262656'],Dishub DKI Nilai Pelabuhan di Jakarta Sudah Penuhi Standar https://t.co/IaV9ptFIrK #RepzoneKalTim #Repzone,Indonesia
13,704.0,1.484938e+09,['id:twitter.com:188050283'],@pulpark this IS ALL Islam they will never Assimilate ever ! Kick out of country one strike your out https://t.co/X5PtfIWsZS,Indonesia
69,2124.0,1.487151e+09,['id:twitter.com:805641344001527809'],"Now Livestreaming Piala Presiden: Gresik United Vs PSS: Jakarta, Persegres Gresik United? https://t.co/52diUrG7Xi",Indonesia
70,2125.0,1.487151e+09,['id:twitter.com:805641344001527809'],"Now Livestreaming Piala Presiden: Gresik United Vs PSS: Jakarta, Persegres Gresik United? https://t.co/52diUrG7Xi https://t.co/bm9C0pqx4h",Indonesia
71,2140.0,1.487169e+09,['id:twitter.com:235845902'],"3 Runner up Terbaik Sementara #PialaPresiden2017 :\n1. Persija Jakarta,\n2. Persegres Gresik United,\n3. Madura United.",Indonesia
73,2142.0,1.487169e+09,['id:twitter.com:781472968035569664'],"3 Runner up Terbaik Sementara #PialaPresiden2017 :\n1. Persija Jakarta,\n2. Persegres Gresik United,\n3. Madura United.",Indonesia
74,2143.0,1.487170e+09,['id:twitter.com:77644078'],"3 Runner up Terbaik Sementara #PialaPresiden2017 :\n1. Persija Jakarta,\n2. Persegres Gresik United,\n3. Madura United.",Indonesia
76,2145.0,1.487170e+09,['id:twitter.com:381911392'],"3 Runner up Terbaik Sementara #PialaPresiden2017 :\n1. Persija Jakarta,\n2. Persegres Gresik United,\n3. Madura United.",Indonesia
77,2146.0,1.487170e+09,['id:twitter.com:1110869071'],"3 Runner up Terbaik Sementara #PialaPresiden2017 :\n1. Persija Jakarta,\n2. Persegres Gresik United,\n3. Madura United.",Indonesia
78,2147.0,1.487171e+09,['id:twitter.com:782083369'],"3 Runner up Terbaik Sementara #PialaPresiden2017 :\n1. Persija Jakarta,\n2. Persegres Gresik United,\n3. Madura United.",Indonesia
